In [1]:
import math
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as tvf
import torchvision.transforms as tvtfms
import operator as op
from PIL import Image
from torch import nn
from timm import create_model

# For type hinting later on
import collections
import typing

In [68]:
net = create_model("convnext_tiny", pretrained=False, num_classes=2, in_chans=3)

In [88]:
# head = nn.Sequential(
#     nn.BatchNorm1d(768),
#     nn.Dropout(0.25),
#     nn.Linear(768, 512, bias=False),
#     nn.ReLU(inplace=True),
#     nn.BatchNorm1d(512),
#     nn.Dropout(0.5),
#     nn.Linear(512, 2, bias=False)
# )

# class CustomHead(nn.Module):
#     def __init__(self):
#         super(CustomHead, self).__init__()
#         self.norm = nn.BatchNorm1d(768)
#         self.fc = nn.Linear(768, 2, bias=True)

#     def forward(self, x):
#         x = self.norm(x)
#         x = self.fc(x)
#         return x

# head = CustomHead()

model = net

In [89]:
# model = nn.Sequential(net, head)

In [90]:
state = torch.load("models/convnext_tiny_best_model.pth")

/tmp/ipykernel_27941/3427640226.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load("models/convnext_tiny_best_model.pth")


In [91]:
list(model.state_dict().keys())

['stem.0.weight',
 'stem.0.bias',
 'stem.1.weight',
 'stem.1.bias',
 'stages.0.blocks.0.gamma',
 'stages.0.blocks.0.conv_dw.weight',
 'stages.0.blocks.0.conv_dw.bias',
 'stages.0.blocks.0.norm.weight',
 'stages.0.blocks.0.norm.bias',
 'stages.0.blocks.0.mlp.fc1.weight',
 'stages.0.blocks.0.mlp.fc1.bias',
 'stages.0.blocks.0.mlp.fc2.weight',
 'stages.0.blocks.0.mlp.fc2.bias',
 'stages.0.blocks.1.gamma',
 'stages.0.blocks.1.conv_dw.weight',
 'stages.0.blocks.1.conv_dw.bias',
 'stages.0.blocks.1.norm.weight',
 'stages.0.blocks.1.norm.bias',
 'stages.0.blocks.1.mlp.fc1.weight',
 'stages.0.blocks.1.mlp.fc1.bias',
 'stages.0.blocks.1.mlp.fc2.weight',
 'stages.0.blocks.1.mlp.fc2.bias',
 'stages.0.blocks.2.gamma',
 'stages.0.blocks.2.conv_dw.weight',
 'stages.0.blocks.2.conv_dw.bias',
 'stages.0.blocks.2.norm.weight',
 'stages.0.blocks.2.norm.bias',
 'stages.0.blocks.2.mlp.fc1.weight',
 'stages.0.blocks.2.mlp.fc1.bias',
 'stages.0.blocks.2.mlp.fc2.weight',
 'stages.0.blocks.2.mlp.fc2.bias',
 '

In [92]:
list(model.state_dict().keys())[-12:]

['stages.3.blocks.2.conv_dw.weight',
 'stages.3.blocks.2.conv_dw.bias',
 'stages.3.blocks.2.norm.weight',
 'stages.3.blocks.2.norm.bias',
 'stages.3.blocks.2.mlp.fc1.weight',
 'stages.3.blocks.2.mlp.fc1.bias',
 'stages.3.blocks.2.mlp.fc2.weight',
 'stages.3.blocks.2.mlp.fc2.bias',
 'head.norm.weight',
 'head.norm.bias',
 'head.fc.weight',
 'head.fc.bias']

In [95]:
list(state.keys())

['stem.0.weight',
 'stem.0.bias',
 'stem.1.weight',
 'stem.1.bias',
 'stages.0.blocks.0.gamma',
 'stages.0.blocks.0.conv_dw.weight',
 'stages.0.blocks.0.conv_dw.bias',
 'stages.0.blocks.0.norm.weight',
 'stages.0.blocks.0.norm.bias',
 'stages.0.blocks.0.mlp.fc1.weight',
 'stages.0.blocks.0.mlp.fc1.bias',
 'stages.0.blocks.0.mlp.fc2.weight',
 'stages.0.blocks.0.mlp.fc2.bias',
 'stages.0.blocks.1.gamma',
 'stages.0.blocks.1.conv_dw.weight',
 'stages.0.blocks.1.conv_dw.bias',
 'stages.0.blocks.1.norm.weight',
 'stages.0.blocks.1.norm.bias',
 'stages.0.blocks.1.mlp.fc1.weight',
 'stages.0.blocks.1.mlp.fc1.bias',
 'stages.0.blocks.1.mlp.fc2.weight',
 'stages.0.blocks.1.mlp.fc2.bias',
 'stages.0.blocks.2.gamma',
 'stages.0.blocks.2.conv_dw.weight',
 'stages.0.blocks.2.conv_dw.bias',
 'stages.0.blocks.2.norm.weight',
 'stages.0.blocks.2.norm.bias',
 'stages.0.blocks.2.mlp.fc1.weight',
 'stages.0.blocks.2.mlp.fc1.bias',
 'stages.0.blocks.2.mlp.fc2.weight',
 'stages.0.blocks.2.mlp.fc2.bias',
 '

In [84]:
list(state.keys())[-8:]

['stages.3.blocks.2.mlp.fc1.weight',
 'stages.3.blocks.2.mlp.fc1.bias',
 'stages.3.blocks.2.mlp.fc2.weight',
 'stages.3.blocks.2.mlp.fc2.bias',
 'head.norm.weight',
 'head.norm.bias',
 'head.fc.weight',
 'head.fc.bias']

In [96]:
# Expected to fail
model.load_state_dict(state)

<All keys matched successfully>

In [80]:
# def copy_weight(name, parameter, state_dict):
#     """
#     Takes in a layer `name`, model `parameter`, and `state_dict`
#     and loads the weights from `state_dict` into `parameter`
#     if it exists.
#     """

#     if name[0] in ["0", "1"]:
#         # name = name[:2] + "model." + name[2:]
#         name = name[2:]
#     if name in state_dict.keys():
#         input_parameter = state_dict[name]
#         if input_parameter.shape == parameter.shape:
#             parameter.copy_(input_parameter)
#         else:
#             print(f'Shape mismatch at layer: {name}, skipping')
#     else:
#         print(f'{name} is not in the state_dict, skipping.')
           
#     # if name[0] == "0":
#     #     name = name[:2] + "model." + name[2:]
#     # if name in state_dict.keys():
#     #     input_parameter = state_dict[name]
#     #     if input_parameter.shape == parameter.shape:
#     #         parameter.copy_(input_parameter)
#     #     else:
#     #         print(f'Shape mismatch at layer: {name}, skipping')
#     # else:
#     #     print(f'{name} is not in the state_dict, skipping.')

In [81]:
# def apply_weights(input_model:nn.Module, input_weights:collections.OrderedDict, application_function:callable):
#     """
#     Takes an input state_dict and applies those weights to the `input_model`, potentially 
#     with a modifier function.
    
#     Args:
#         input_model (`nn.Module`):
#             The model that weights should be applied to
#         input_weights (`collections.OrderedDict`):
#             A dictionary of weights, the trained model's `state_dict()`
#         application_function (`callable`):
#             A function that takes in one parameter and layer name from `input_model`
#             and the `input_weights`. Should apply the weights from the state dict into `input_model`.
#     """
#     model_dict = input_model.state_dict()
#     for name, parameter in model_dict.items():
#         application_function(name, parameter, input_weights)
#     input_model.load_state_dict(model_dict)

In [93]:
# apply_weights(model, state, copy_weight)

In [62]:
import torch
from torchvision import transforms
from PIL import Image

# define the labels as the came from dls.vocab
class_labels = ['no', 'yes']

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the size expected by the model
    transforms.ToTensor(),          # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Load the image
image_path = 'processed/valid/yes/Y1.jpg'
image = Image.open(image_path)
image = transform(image).unsqueeze(0)  # Add a batch dimension

model.eval()  # Set the model to evaluation mode

# Run the image through the model
with torch.no_grad():
    output = model(image)

# Get the predicted class
_, predicted_class = torch.max(output, 1)
predicted_index = predicted_class.item()

# Map the index to the class label
predicted_label = class_labels[predicted_index]

print(f'Predicted class index: {predicted_index}')
print(f'Predicted class label: {predicted_label}')

Predicted class index: 1
Predicted class label: yes


In [94]:
import torch
from torchvision import transforms
from PIL import Image
import os

# Define the labels as they came from dls.vocab
class_labels = ['no', 'yes']

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the size expected by the model
    transforms.ToTensor(),          # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Path to the "no" folder
folder_path = 'processed/valid/no'

# List all image files in the "no" folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

model.eval()  # Set the model to evaluation mode

# Loop through each image file
for image_file in image_files:
    print(f'Processing Image: {image_file}')
    image_path = os.path.join(folder_path, image_file)
    image = Image.open(image_path)
    
    print(f'Image shape: {image.size}')
    # print no of channels
    print(f'Image channels: {len(image.getbands())}')
    
    # Convert single-channel images to three-channel
    if image.mode != 'RGB':
        image = image.convert('RGB')

    image = transform(image).unsqueeze(0)  # Add a batch dimension

    # Run the image through the model
    with torch.no_grad():
        output = model(image)

    # Get the predicted class
    _, predicted_class = torch.max(output, 1)
    predicted_index = predicted_class.item()

    # Map the index to the class label
    predicted_label = class_labels[predicted_index]

    print(f'Image: {image_file}')
    print(f'Predicted class index: {predicted_index}')
    print(f'Predicted class label: {predicted_label}')
    print('---')

Processing Image: 25 no.jpg
Image shape: (232, 217)
Image channels: 3
Image: 25 no.jpg
Predicted class index: 0
Predicted class label: no
---
Processing Image: 28 no.jpg
Image shape: (201, 251)
Image channels: 3
Image: 28 no.jpg
Predicted class index: 1
Predicted class label: yes
---
Processing Image: 33 no.jpg
Image shape: (236, 213)
Image channels: 3
Image: 33 no.jpg
Predicted class index: 0
Predicted class label: no
---
Processing Image: N1.JPG
Image shape: (276, 338)
Image channels: 3
Image: N1.JPG
Predicted class index: 1
Predicted class label: yes
---
Processing Image: N15.jpg
Image shape: (225, 225)
Image channels: 1
Image: N15.jpg
Predicted class index: 0
Predicted class label: no
---
Processing Image: no 4.jpg
Image shape: (220, 275)
Image channels: 1
Image: no 4.jpg
Predicted class index: 0
Predicted class label: no
---
Processing Image: no 92.jpg
Image shape: (206, 244)
Image channels: 3
Image: no 92.jpg
Predicted class index: 0
Predicted class label: no
---
Processing Image